In [5]:
import numpy as np
from numpy.lib.format import open_memmap
import myfunctions as mf
import myplots as mp
import os
import pandas as pd
from tqdm import tqdm
import napari

In [26]:
def take(hypervolume_mask, lookup_table):
    hypervolume_mask = np.take(lookup_table, hypervolume_mask)
    return None

In [33]:
def take2(hypervolume_mask, lookup_table):
    hypervolume_mask[0,0] = 3
    print(np.take(lookup_table, hypervolume_mask))
    hypervolume_mask[:] = np.take(lookup_table, hypervolume_mask)
    return None

In [34]:
exp='P28A_FT_H_Exp2'
OS='MacOS_SSD'
hypervolume_mask = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume_mask_test.npy'), mode='w+', dtype='float32', shape=(2, 2))
hypervolume_mask = np.array([[0, 1], [2, 3]])
print(hypervolume_mask)
lookup_table = np.array([[0, 1], [0, 3]])
hypervolume_mask = np.take(lookup_table, hypervolume_mask)
print('take')
print(hypervolume_mask)
hypervolume_mask = np.array([[0, 1], [2, 3]])
take(hypervolume_mask, lookup_table)
print('take fun')
print(hypervolume_mask)
hypervolume_mask = np.array([[0, 1], [2, 3]])
take2(hypervolume_mask, lookup_table)
print('take2 fun')
print(hypervolume_mask)

[[0 1]
 [2 3]]
take
[[0 1]
 [0 3]]
take fun
[[0 1]
 [2 3]]
[[3 1]
 [0 3]]
take2 fun
[[3 1]
 [0 3]]


In [7]:
exp='P28A_FT_H_Exp2'
OS='MacOS_SSD'
hypervolume_mask = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume_mask.npy'), mode='r+')
max_label = np.max(hypervolume_mask)
start_TR = mf.exp_start_TR_list()[mf.exp_list().index(exp)]
labels_to_remove = np.union1d(np.setdiff1d(np.unique(hypervolume_mask[:start_TR]), np.array([0, 1])),
                                np.array(mf.labels_to_remove_list()[mf.exp_list().index(exp)])).astype(np.ushort)
print(labels_to_remove)

[ 2  8 12 34 51 53 68 71 86]


In [9]:
lookup_table = np.arange(max_label+1)
lookup_table[labels_to_remove] = 0
print(lookup_table[:12])

[ 0  1  0  3  4  5  6  7  0  9 10 11]


In [14]:
take(hypervolume_mask, lookup_table)

In [4]:
hypervolume_mask = np.take(lookup_table, hypervolume_mask)

In [6]:
hypervolume = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume.npy'), mode='r')
viewer = napari.Viewer()

images = [viewer.add_image(hypervolume, name='Volume', opacity=0.4)] 
labels = [viewer.add_labels(hypervolume_mask, name='Mask', opacity=0.8)]

settings = napari.settings.get_settings()
settings.application.playback_fps = 5
viewer.dims.current_step = (0, 0)

In [15]:
exp='P28A_FT_H_Exp1'
OS='MacOS_SSD'
df = pd.read_csv(os.path.join(mf.OS_path(exp, OS), 'motion_properties.csv'))
c = df.copy()

In [18]:
c.head()

,t,label,x,y,z,r,vx,vy,vxy,vz,v,V,dVdt,r_sect,z_sect
0,0.00,2,1.152986,0.394468,5.142758,1.218598,0.000000,0.000000,0.000000,0.000000,0.000000,3.132352,62.64704,Core,Middle
1,0.00,10,-0.373433,-0.214925,10.252537,0.430865,0.000000,0.000000,0.000000,0.000000,0.000000,0.008576,0.17152,Core,Top
2,0.05,1,1.414619,0.496106,5.279259,1.499089,5.232664,2.032752,5.613631,2.730027,6.242267,3.252160,2.39616,Core,Middle
3,0.05,5,-0.373178,-0.162326,10.212093,0.406954,0.005091,1.051996,1.052008,-0.808886,1.327033,0.008256,-0.00640,Core,Top
4,0.05,27,-7.414432,5.061026,3.302564,8.977070,0.000000,0.000000,0.000000,0.000000,0.000000,0.017472,0.34944,External,Bottom


In [17]:
# temp = c.loc[c['t'] == 0.05, 'label']
c.loc[c['t'] == 0.05, 'label'] = c.loc[c['t'] == 0.05, 'label'] / 2

In [3]:
exp_list = ['P28A_FT_H_Exp4_2', 'P28B_ISC_FT_H_Exp2', 'VCT5_FT_N_Exp1']
crop_list = [100, 50, 100]
parent_dir = '/Volumes/T7/Thesis/'

In [4]:
for exp, crop in zip(exp_list, crop_list):
    exp_dir = os.path.join(parent_dir, exp)
    v1 = open_memmap(os.path.join(exp_dir, 'hypervolume.npy'), mode='r')
    t = v1.shape[0]
    z = v1.shape[1]
    xy = v1.shape[2]
    v2 = open_memmap(os.path.join(exp_dir, 'v2.npy'), mode='w+', dtype=np.half, shape=(t, z, xy-(2*crop), xy-(2*crop)))
    for time in tqdm(range(t), desc=exp):
        v2[time] = v1[time, :, crop:v1.shape[2]-crop, crop:v1.shape[2]-crop]

VCT5_FT_N_Exp1: 100%|██████████| 33/33 [02:05<00:00,  3.81s/it]
